In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
os.sys.path.insert(0, '/home/schirrmr/code/explaining/reversible//')
os.sys.path.insert(0, '/home/schirrmr/code/reversible/reversible2///')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible.sliced import sliced_from_samples

from numpy.random import RandomState

import torch as th
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
from reversible.plot import create_bw_image
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np, set_random_seeds

In [ ]:
class UpperBoundReLU(nn.Module):
    def __init__(self, n_filters):
        super(UpperBoundReLU, self).__init__()
        self.upper_bound = nn.Parameter(th.ones(n_filters, dtype=th.float32))
        #self.upper_bound = th.ones(n_filters, dtype=th.float32)
        
    def forward(self, x):
        x = nn.functional.relu(x)
        mask = (x < self.upper_bound.unsqueeze(0)).type_as(x)
        x = x * mask + ((1-mask ) * self.upper_bound.unsqueeze(0))
        return x

In [ ]:
example_points = np_to_var([0,1.0,2.0],dtype=np.float32).unsqueeze(1).detach().requires_grad_(True)

In [ ]:
set_random_seeds(20190118, False)
discriminator = nn.Sequential(nn.Linear(1,60), 
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,60),
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,1))

from spectral_norm import spectral_norm

#for module in discriminator.modules():
#    if hasattr(module, 'weight'):
#        spectral_norm(module,n_power_iterations=3)

In [ ]:
counter_points = np_to_var(np.linspace(-0.5,2.5,4), dtype=np.float32).unsqueeze(1)

plt.plot(var_to_np(example_points), var_to_np(example_points) * 0, ls='', marker='o')
plt.plot(var_to_np(counter_points), var_to_np(counter_points) * 0, ls='', marker='o')

In [ ]:
sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

for i_module in range(len(list(discriminator.children()))):
    outs = discriminator[:i_module+1](sorted_p)
    plt.figure(figsize=(6,1.2))
    plt.plot(var_to_np(sorted_p), var_to_np(outs[:,:10]), color='black')
    print(outs.shape)

In [ ]:
out

In [ ]:
optimizer = th.optim.Adam(discriminator.parameters(), lr=1e-2)
n_epochs = 5000
for i_epoch in range(n_epochs):
    out = th.mean(discriminator(example_points)) - th.mean(discriminator(counter_points))
    l1 = example_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(example_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("loss", loss)
        sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

        outs = discriminator(sorted_p)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(sorted_p.squeeze()), var_to_np(outs))
        display(fig)
        plt.close(fig)

In [ ]:
optimizer = th.optim.Adam(discriminator.parameters(), lr=1e-2)
n_epochs = 5000
for i_epoch in range(n_epochs):
    out = th.mean(discriminator(example_points)) - th.mean(discriminator(counter_points))
    l1 = example_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(example_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        print("loss", loss)
        sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

        outs = discriminator(sorted_p)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(sorted_p.squeeze()), var_to_np(outs))
        display(fig)
        plt.close(fig)

In [ ]:
out

In [ ]:
sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

for i_module in range(len(list(discriminator.children()))):
    outs = discriminator[:i_module+1](sorted_p)
    plt.figure(figsize=(6,1.2))
    plt.plot(var_to_np(sorted_p), var_to_np(outs[:,:]), color='black', lw=0.5)
    print(outs.shape)

## only use first and last point

In [ ]:
from weight_norm import weight_norm
set_random_seeds(20190118, False)
discriminator = nn.Sequential(nn.Linear(1,60), 
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,60),
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,1))


weight_norms = []
for m in discriminator.modules():
    if hasattr(m, 'weight') and m.weight is not None:
        w_norms = th.ones_like(m.weight[:,0:1], requires_grad=True)
        w_norms.data = w_norms.data *  1
        # in-place!!
        print("wnorm", w_norms.shape)
        weight_norm(m, 'weight', dim=0, fixed_norm=w_norms)
        weight_norms.append(w_norms)
None

In [ ]:
optimizer = th.optim.Adam(discriminator.parameters(), lr=1e-2)
optim_wnorms = th.optim.Adam(weight_norms, lr=1e-3)
n_epochs = 20
for i_epoch in range(n_epochs):
    used_points = example_points[[0,2]]
    out = th.mean(discriminator(used_points)) - th.mean(discriminator(counter_points))
    l1 = used_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(used_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    used_points = example_points[[1,]]
    out = th.mean(discriminator(used_points)) - th.mean(discriminator(counter_points))
    l1 = used_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(used_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optim_wnorms.zero_grad()
    loss.backward()
    optim_wnorms.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        print("loss", loss)
        sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

        outs = discriminator(sorted_p)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(sorted_p.squeeze()), var_to_np(outs))
        display(fig)
        plt.close(fig)

In [ ]:
from weight_norm import weight_norm
set_random_seeds(20190118, False)
discriminator = nn.Sequential(nn.Linear(1,60), 
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,60),
                             nn.ReLU(),#UpperBoundReLU(20)
                             nn.Linear(60,1))


weight_norms = []
for m in discriminator.modules():
    if hasattr(m, 'weight') and m.weight is not None:
        w_norms = th.ones_like(m.weight[:,0:1], requires_grad=True)
        w_norms.data = w_norms.data *  1
        # in-place!!
        print("wnorm", w_norms.shape)
        weight_norm(m, 'weight', dim=0, fixed_norm=w_norms)
        weight_norms.append(w_norms)
None

In [ ]:
a = th.randn(784,2000)

In [ ]:
%%timeit
_, _ = th.sort(a, dim=0)

In [ ]:
optimizer = th.optim.Adam(discriminator.parameters(), lr=1e-2)
optim_wnorms = th.optim.Adam(weight_norms, lr=1e-3)
n_epochs = 2000
for i_epoch in range(n_epochs):
    used_points = example_points[th.randperm(len(example_points))[:2]]
    out = th.mean(discriminator(used_points)) - th.mean(discriminator(counter_points))
    l1 = used_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(used_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    used_points = example_points[th.randperm(len(example_points))[:1]]
    out = th.mean(discriminator(used_points)) - th.mean(discriminator(counter_points))
    l1 = used_points.unsqueeze(1)- counter_points.unsqueeze(0)
    dist = discriminator(used_points).unsqueeze(1) - discriminator(counter_points).unsqueeze(0)
    dist_loss = nn.functional.relu(th.abs(dist) - th.abs(l1))
    dist_loss =  dist_loss + (dist_loss ** 2 )
    loss = -out + th.mean(dist_loss) * 10
    optim_wnorms.zero_grad()
    loss.backward()
    optim_wnorms.step()
    
    if i_epoch % (n_epochs // 20) == 0:
        print("loss", loss)
        sorted_p, _ = th.sort(th.cat((example_points, counter_points)), dim=0)

        outs = discriminator(sorted_p)
        fig = plt.figure(figsize=(8,3))
        plt.plot(var_to_np(sorted_p.squeeze()), var_to_np(outs))
        display(fig)
        plt.close(fig)

In [ ]:
import ot

In [ ]:
used_points.shape

In [ ]:
distances = np.sum(np.abs(var_to_np(used_points)[:,None] - var_to_np(counter_points)[None]), axis=(2))
coupling = ot.emd([],[], distances)

In [ ]:
np.sum(coupling * distances)

In [ ]:
distac